Get all cemeteries in Wikidata and check OSM
* [This Notebook](https://github.com/salgo60/Gravstensinventeringen-Wikidata/blob/main/Notebook/OSM%20Wikidata%20cemeteryGermany.ipynb)
* [Video](https://youtu.be/EDkPoomRmJg)


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-02-15 11:08:21.760741


In [2]:
import pandas as pd
#
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
 
# https://w.wiki/6Lcb
queryAllCem = """#title cemeteries in WD
SELECT ?item (REPLACE(STR(?item), ".*Q", "Q") AS ?itemid) ?itemLabel ?OSM ?OSMrel WHERE {
  #?item wdt:P6104 wd:Q115206846.
  ?item wdt:P31 wd:Q39614.
  {?item wdt:P17 wd:Q183}
  OPTIONAL {?item wdt:P10689 ?OSM}
  OPTIONAL {?item wdt:P402 ?OSMrel}

  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY (?itemLabel)"""


def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDCem = get_sparql_dataframe(endpoint_url, queryAllCem)    
WDCem.shape

(7499, 5)

In [3]:
WDCem.head()

,item,itemid,itemLabel,OSM,OSMrel
0,http://www.wikidata.org/entity/Q100697844,Q100697844,Q100697844,None,None
1,http://www.wikidata.org/entity/Q101103852,Q101103852,Q101103852,None,None
2,http://www.wikidata.org/entity/Q101103853,Q101103853,Q101103853,None,None
3,http://www.wikidata.org/entity/Q101103854,Q101103854,Q101103854,None,None
4,http://www.wikidata.org/entity/Q101103855,Q101103855,Q101103855,None,None


In [4]:
import urllib3, json
from tqdm import tqdm
http = urllib3.PoolManager()

listWD = []
for WD, row in tqdm(WDCem.iterrows(), total=WDCem.shape[0]):
    #print(row["itemid"] ) 
    url = "https://osm.wikidata.link/tagged/api/item/" + row["itemid"] 
    
    new_item = dict()
    new_item['wikidata'] = row["itemid"] 
#    new_item['coord'] = row["coord"] 
    try:
        r = http.request('GET', url) 
        data = json.loads(r.data.decode('utf-8'))
    except:
        print (r.status, url)
#    print (r.status)
    try:
        #print ("OSM ", data["osm"], "Type: ", type(data["osm"]))
        #print ("ID: ", data["osm"][0]["id"] , "\tType: ", data["osm"][0]["type"])
        osmid = data["osm"][0]["id"] 
        osmType = data["osm"][0]["type"]
        new_item['osmid'] = osmid 
        new_item['type'] = osmType
        listWD.append(new_item)
        #print(listWD)

    except:
        #print ("error")
        pass
print (len(listWD))

100%|██████████| 7499/7499 [14:25<00:00,  8.66it/s]  

896


In [5]:
listWD

[{'wikidata': 'Q101195218', 'osmid': 225451958, 'type': 'way'},
 {'wikidata': 'Q101209099', 'osmid': 25969473, 'type': 'way'},
 {'wikidata': 'Q101248105', 'osmid': 211436344, 'type': 'way'},
 {'wikidata': 'Q101277539', 'osmid': 101537265, 'type': 'way'},
 {'wikidata': 'Q101279148', 'osmid': 545291453, 'type': 'way'},
 {'wikidata': 'Q101403211', 'osmid': 490379888, 'type': 'way'},
 {'wikidata': 'Q102180272', 'osmid': 36808592, 'type': 'way'},
 {'wikidata': 'Q102180278', 'osmid': 96750427, 'type': 'way'},
 {'wikidata': 'Q102180280', 'osmid': 361226726, 'type': 'way'},
 {'wikidata': 'Q104228316', 'osmid': 90882573, 'type': 'way'},
 {'wikidata': 'Q104395987', 'osmid': 173908614, 'type': 'way'},
 {'wikidata': 'Q104533878', 'osmid': 179358264, 'type': 'way'},
 {'wikidata': 'Q104831696', 'osmid': 640257991, 'type': 'way'},
 {'wikidata': 'Q105621785', 'osmid': 28453895, 'type': 'way'},
 {'wikidata': 'Q105622507', 'osmid': 23600771, 'type': 'way'},
 {'wikidata': 'Q106090708', 'osmid': 492766737

In [6]:
OSMtot = pd.DataFrame(listWD,
                  columns=['wikidata','osmid','type'])
OSMtot.shape

(896, 3)

In [7]:
pd.set_option('max_colwidth', 400)
OSMtot.head(10)

,wikidata,osmid,type
0,Q101195218,225451958,way
1,Q101209099,25969473,way
2,Q101248105,211436344,way
3,Q101277539,101537265,way
4,Q101279148,545291453,way
5,Q101403211,490379888,way
6,Q102180272,36808592,way
7,Q102180278,96750427,way
8,Q102180280,361226726,way
9,Q104228316,90882573,way


In [8]:
OSMtot.to_csv("WD_OSM_Germany_Cemetary.csv")

* [WD_OSM_Germany_Cemetary.csv](https://github.com/salgo60/Gravstensinventeringen-Wikidata/blob/main/Notebook/WD_OSM_Germany_Cemetary.csv)

In [9]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2023-02-15 11:23:14.522661
Time elapsed (hh:mm:ss.ms) 0:14:52.762847
